In [1]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  bzip2-doc libboost-atomic1.71-dev libboost-atomic1.71.0
  libboost-chrono1.71-dev libboost-chrono1.71.0 libboost-date-time1.71-dev
  libboost-date-time1.71.0 libboost-program-options1.71-dev
  libboost-program-options1.71.0 libboost-serialization1.71-dev
  libboost-serialization1.71.0 libboost-system1.71-dev libboost-system1.71.0
  libboost-test1.71-dev libboost-test1.71.0 libboost-thread1.71-dev
  libboost-thread1.71.0 libboost1.71-dev
Suggested packages:
  libboost1.71-doc libboost-container1.71-dev libboost-context1.71-dev
  libboost-contract1.71-dev libboost-coroutine1.71-dev
  libboost-exception1.71-dev 

before downloading and unpacking the KenLM repo.

In [2]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2024-01-27 18:39:08--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  2.21MB/s    in 0.2s    

2024-01-27 18:39:08 (2.21 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [3]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZi

Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [5]:

!kenlm/build/bin/lmplz -o 5 <"dafyomi_clean.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /teamspace/studios/this_studio/FinalProject/word2vec-kenlm/dafyomi_clean.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100


****************************************************************************************************
Unigram tokens 913797 types 38453
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:461436 2:1270567808 3:2382315008 4:3811703552 5:5558734848
Statistics:
1 38453 D1=0.588645 D2=1.06547 D3+=1.53031
2 370354 D1=0.769053 D2=1.17374 D3+=1.46219
3 697019 D1=0.886361 D2=1.32185 D3+=1.50432
4 829955 D1=0.952207 D2=1.47275 D3+=1.64234
5 867607 D1=0.961877 D2=1.64998 D3+=1.6186
Memory estimate for binary LM:
type    MB
probing 59 assuming -p 1.5
probing 70 assuming -r models -p 1.5
trie    28 without quantization
trie    15 assuming -q 8 -b 8 quantization 
trie    25 assuming -a 22 array pointer compression
trie    12 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:461436 2:5925664 3:13940380 4:19918920 5:24292996
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [6]:
!head -20 5gram.arpa

\data\
ngram 1=38453
ngram 2=370354
ngram 3=697019
ngram 4=829955
ngram 5=867607

\1-grams:
-5.5554833	<unk>	0
0	<s>	-0.9983632
-2.176712	</s>	0
-3.9402242	שלום	-0.14760421
-3.8484545	לכולם	-0.18955754
-2.6926877	אנחנו	-0.6608004
-3.324339	לומדים	-0.38266155
-1.9637972	את	-0.6789234
-4.129278	דף	-0.18102482
-4.755677	קט	-0.17820328
-3.349818	במסכת	-0.8243355
-4.1620703	חולין	-0.15478066


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [7]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [8]:
!head -20 5gram_correct.arpa

\data\
ngram 1=38454
ngram 2=370354
ngram 3=697019
ngram 4=829955
ngram 5=867607

\1-grams:
-5.5554833	<unk>	0
0	<s>	-0.9983632
0	</s>	-0.9983632
-2.176712	</s>	0
-3.9402242	שלום	-0.14760421
-3.8484545	לכולם	-0.18955754
-2.6926877	אנחנו	-0.6608004
-3.324339	לומדים	-0.38266155
-1.9637972	את	-0.6789234
-4.129278	דף	-0.18102482
-4.755677	קט	-0.17820328
-3.349818	במסכת	-0.8243355


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.